# Detailed Music Collection Analysis

This notebook extracts detailed information from a Traktor collection.nml file and organizes it into a pandas DataFrame. The extracted information includes:

- Artist
- Title
- Label
- Album/Release
- Release Date
- Comments

The data is then organized and analyzed to provide insights into the music collection.

## 1. Import Required Libraries

In this section, we'll import the necessary libraries for working with XML data and creating DataFrames.

In [1]:
# Import required libraries
import xml.etree.ElementTree as ET
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os

# Set some display options for pandas
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 20)       # Limit rows for better readability
pd.set_option('display.width', 1000)        # Wider display for better viewing
pd.set_option('display.colheader_justify', 'left')  # Left-align column headers

# Check for the collection.nml file
file_path = '../data/collection.nml'
print(f"File exists: {os.path.exists(file_path)}")
print(f"File size: {os.path.getsize(file_path) / (1024*1024):.2f} MB")

File exists: True
File size: 4.02 MB


## 2. Define Song Data Structure

Before we parse the XML file, let's define the structure for our song data. We'll create a function to extract the detailed information from each entry in the collection.

In [2]:
def extract_song_data(entry):
    """
    Extract detailed song information from an XML entry element
    
    Parameters:
    entry (Element): XML Element representing a song entry
    
    Returns:
    dict: Dictionary containing the extracted song information
    """
    # Initialize song data with empty values
    song_data = {
        'Artist': '',
        'Title': '',
        'Album': '',
        'Label': '',
        'Release Date': '',
        'Comment': '',
        'Genre': '',
        'BPM': '',
        'Key': '',
        'Play Count': 0,
        'File Path': '',
        'Duration': 0,
        'File Size': 0,
        'Last Played': ''
    }
    
    # Extract basic information from the entry attributes
    song_data['Artist'] = entry.get('ARTIST', '')
    song_data['Title'] = entry.get('TITLE', '')
    
    # Extract album information
    album_elem = entry.find('./ALBUM')
    if album_elem is not None:
        song_data['Album'] = album_elem.get('TITLE', '')
    
    # Extract detailed information from INFO element
    info_elem = entry.find('./INFO')
    if info_elem is not None:
        song_data['Label'] = info_elem.get('LABEL', '')
        song_data['Comment'] = info_elem.get('COMMENT', '')
        song_data['Genre'] = info_elem.get('GENRE', '')
        song_data['Key'] = info_elem.get('KEY', '')
        song_data['Play Count'] = int(info_elem.get('PLAYCOUNT', 0))
        song_data['Duration'] = float(info_elem.get('PLAYTIME', 0))
        song_data['File Size'] = float(info_elem.get('FILESIZE', 0))
        
        # Convert release date from string to datetime if available
        release_date = info_elem.get('RELEASE_DATE', '')
        if release_date:
            # Format is typically YYYY/M/D
            try:
                year, month, day = release_date.split('/')
                song_data['Release Date'] = f"{year}-{month.zfill(2)}-{day.zfill(2)}"
            except:
                song_data['Release Date'] = release_date
        
        # Get last played date if available
        last_played = info_elem.get('LAST_PLAYED', '')
        if last_played:
            song_data['Last Played'] = last_played
    
    # Extract BPM information
    tempo_elem = entry.find('./TEMPO')
    if tempo_elem is not None:
        song_data['BPM'] = tempo_elem.get('BPM', '')
    
    # Extract file path information
    location_elem = entry.find('./LOCATION')
    if location_elem is not None:
        dir_path = location_elem.get('DIR', '').replace('/', os.sep)
        file_name = location_elem.get('FILE', '')
        song_data['File Path'] = os.path.join(dir_path, file_name)
    
    return song_data

# Create a simple function to check if any of the required fields are missing
def has_required_fields(song_data):
    """
    Check if a song has the required fields
    
    Parameters:
    song_data (dict): Dictionary containing song information
    
    Returns:
    bool: True if all required fields are present, False otherwise
    """
    required_fields = ['Artist', 'Title']
    return all(song_data[field] for field in required_fields)

## 3. Extract Song Information

Now, let's parse the XML file and extract the song information using the functions we defined.

In [3]:
# Parse the XML file
try:
    tree = ET.parse(file_path)
    root = tree.getroot()
    print(f"Successfully parsed XML file")
    print(f"Root tag: {root.tag}")
    print(f"Root attributes: {root.attrib}")
except Exception as e:
    print(f"Error parsing XML file: {e}")
    raise

# Find all ENTRY elements
entries = root.findall('.//ENTRY')
print(f"Number of entries found: {len(entries)}")

# Extract song data from each entry
songs = []
for entry in entries:
    song_data = extract_song_data(entry)
    if has_required_fields(song_data):
        songs.append(song_data)

print(f"Successfully extracted data for {len(songs)} songs")

# Display the first entry as an example
if songs:
    print("\nExample of extracted song data:")
    for key, value in songs[0].items():
        print(f"{key}: {value}")

Successfully parsed XML file
Root tag: NML
Root attributes: {'VERSION': '19'}
Number of entries found: 4151
Successfully extracted data for 3003 songs

Example of extracted song data:
Artist: aaliyah
Title: try again (a cappella)
Album: try again (vinyl single)
Label: 
Release Date: 
Comment: 
Genre: A Cappella A capella
BPM: 96.212700
Key: 1m
Play Count: 0
File Path: /:Users/:roel4ez/:Music/:iTunes-Traktor/:music/:Music/:aaliyah/:try again (vinyl single)/:/try again (a cappella).mp3
Duration: 268.0
File Size: 6540.0
Last Played: 


## 4. Create DataFrame

Now, let's convert our list of song dictionaries into a pandas DataFrame for easier manipulation and analysis.

In [4]:
# Create a DataFrame from the extracted song data
songs_df = pd.DataFrame(songs)

# Check the shape of the DataFrame
print(f"DataFrame shape: {songs_df.shape}")

# Convert numeric columns to appropriate types
numeric_columns = ['Play Count', 'Duration', 'File Size']
for col in numeric_columns:
    if col in songs_df.columns:
        songs_df[col] = pd.to_numeric(songs_df[col], errors='coerce')

# Convert date columns to datetime type
date_columns = ['Release Date', 'Last Played']
for col in date_columns:
    if col in songs_df.columns:
        songs_df[col] = pd.to_datetime(songs_df[col], errors='coerce')

# Convert BPM to float
if 'BPM' in songs_df.columns:
    songs_df['BPM'] = pd.to_numeric(songs_df['BPM'], errors='coerce')

# Display DataFrame information
print("\nDataFrame Information:")
songs_df.info()

# Display DataFrame column statistics
print("\nNumeric Column Statistics:")
print(songs_df.describe())

DataFrame shape: (3003, 14)

DataFrame Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3003 entries, 0 to 3002
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Artist        3003 non-null   object        
 1   Title         3003 non-null   object        
 2   Album         3003 non-null   object        
 3   Label         3003 non-null   object        
 4   Release Date  2483 non-null   datetime64[ns]
 5   Comment       3003 non-null   object        
 6   Genre         3003 non-null   object        
 7   BPM           2828 non-null   float64       
 8   Key           3003 non-null   object        
 9   Play Count    3003 non-null   int64         
 10  File Path     3003 non-null   object        
 11  Duration      3003 non-null   float64       
 12  File Size     3003 non-null   float64       
 13  Last Played   1876 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(3), in

## 5. Display and Analyze Data

Now that we have our data in a DataFrame, let's explore and analyze it.

In [5]:
# Display the first few rows of the DataFrame
print("First few rows of the DataFrame:")
display(songs_df.head())

# Display the last few rows of the DataFrame
print("\nLast few rows of the DataFrame:")
display(songs_df.tail())

# Check for missing values
print("\nMissing values per column:")
missing_values = songs_df.isnull().sum()
print(missing_values[missing_values > 0])

# Core information columns we're interested in
core_columns = ['Artist', 'Title', 'Album', 'Label', 'Release Date', 'Comment', 'Genre']
print(f"\nSample of core columns ({', '.join(core_columns)}):")
display(songs_df[core_columns].head(10))

First few rows of the DataFrame:


,Artist,Title,Album,Label,Release Date,Comment,Genre,BPM,Key,Play Count,File Path,Duration,File Size,Last Played
0,aaliyah,try again (a cappella),try again (vinyl single),,NaT,,A Cappella A capella,96.212700,1m,0,/:Users/:roel4ez/:Music/:iTunes-Traktor/:music...,268.0,6540.0,NaT
1,Absolute Zero & Subphonics,The Code,Hardware XV - History Of Hardware CD1,Renegade Hardware,1999-01-01,Vinyl,Drum & Bass,170.192902,4m,3,/:Users/:roel4ez/:Music/:iTunes-Traktor/:music...,436.0,17016.0,2025-03-21
2,Adam F,Brand New Funk,Drum & Bass Arena - 20 Years of Drum N Bass 19...,,2016-01-01,Vinyl,Drum & Bass,171.291626,5m,5,/:Users/:roel4ez/:Music/:iTunes-Traktor/:music...,466.0,18216.0,2021-02-27
3,Alix Perez,Down The Line (feat. MC Fats),10 Years Of Shogun Audio,Shogun Audio,2014-01-01,,Drum & Bass,171.996994,10m,4,/:Users/:roel4ez/:Music/:iTunes-Traktor/:music...,372.0,14780.0,2020-01-03
4,Alix Perez,Fade Away,1984,,2009-01-01,,Drum & Bass,172.999741,1m,6,/:Users/:roel4ez/:Music/:iTunes-Traktor/:music...,339.0,13436.0,2022-06-24



Last few rows of the DataFrame:


,Artist,Title,Album,Label,Release Date,Comment,Genre,BPM,Key,Play Count,File Path,Duration,File Size,Last Played
2998,Surge,Cisco,Through The Eyes,Full Cycle,2000-01-01,,Drum & Bass,NaN,,0,/:Users/:roel4ez/:Music/:iTunes-Traktor/:music...,347.0,13636.0,NaT
2999,Die,Jitta Bug (Remix),Through The Eyes,Full Cycle,2000-01-01,,Drum & Bass,NaN,,0,/:Users/:roel4ez/:Music/:iTunes-Traktor/:music...,398.0,15620.0,NaT
3000,Suv,Dark Angel,Through The Eyes,Full Cycle,2000-01-01,,Drum & Bass,NaN,,0,/:Users/:roel4ez/:Music/:iTunes-Traktor/:music...,381.0,14956.0,NaT
3001,Roni Size,Breaks,Through The Eyes,Full Cycle,2000-01-01,,Drum & Bass,NaN,,0,/:Users/:roel4ez/:Music/:iTunes-Traktor/:music...,354.0,13900.0,NaT
3002,Calyx,Follow The Leader (featuring Teebee) (ASHADOW3...,,Moving Shadow,NaT,www.mediahuman.com,,171.986618,7m,0,/:Users/:roel4ez/:Music/:iTunes-Traktor/:music...,384.0,15208.0,NaT



Missing values per column:
Release Date     520
BPM              175
Last Played     1127
dtype: int64

Sample of core columns (Artist, Title, Album, Label, Release Date, Comment, Genre):


,Artist,Title,Album,Label,Release Date,Comment,Genre
0,aaliyah,try again (a cappella),try again (vinyl single),,NaT,,A Cappella A capella
1,Absolute Zero & Subphonics,The Code,Hardware XV - History Of Hardware CD1,Renegade Hardware,1999-01-01,Vinyl,Drum & Bass
2,Adam F,Brand New Funk,Drum & Bass Arena - 20 Years of Drum N Bass 19...,,2016-01-01,Vinyl,Drum & Bass
3,Alix Perez,Down The Line (feat. MC Fats),10 Years Of Shogun Audio,Shogun Audio,2014-01-01,,Drum & Bass
4,Alix Perez,Fade Away,1984,,2009-01-01,,Drum & Bass
5,Alix Perez,Forsaken feat. Peven Everett & SpectraSoul,1984,,2009-01-01,,Drum & Bass
6,Alix Perez,Revolve-Her,Recall and Reflect EP-(EXIT059),,2015-01-01,,Drum & Bass
7,Alix Perez,Never Left,Recall and Reflect EP-(EXIT059),,2015-01-01,,Drum & Bass
8,Alix Perez,The Cut Deepens ft. Foreign Beggars,,,NaT,,Drum & Bass
9,Ancronix,Skin it Back,320 DUB,,2004-01-01,00000E1B 00000DDB 00004323 00003E79 0005093E 0...,Drum & Bass


In [8]:
# Load the gazmazk4ez collection
collection_path = '../data/gazmazk4ez-collection-20250608-1029.csv'
try:
    gazmazk4ez_df = pd.read_csv(collection_path)
    print(f"Successfully loaded gazmazk4ez collection with {len(gazmazk4ez_df)} entries")
except Exception as e:
    print(f"Error loading gazmazk4ez collection: {e}")
    raise

# Inspect the gazmazk4ez collection
print("\nColumns in gazmazk4ez collection:")
print(gazmazk4ez_df.columns)

# Add a flag to the songs DataFrame
songs_df['In Gazmazk4ez Collection'] = songs_df['Title'].isin(gazmazk4ez_df['Title'])

# Display the updated DataFrame
print("\nUpdated DataFrame with Gazmazk4ez flag:")
display(songs_df[['Artist', 'Title', 'In Gazmazk4ez Collection']].head(10))

Successfully loaded gazmazk4ez collection with 598 entries

Columns in gazmazk4ez collection:
Index(['Catalog#', 'Artist', 'Title', 'Label', 'Format', 'Rating', 'Released', 'release_id', 'CollectionFolder', 'Date Added', 'Collection Media Condition', 'Collection Sleeve Condition', 'Collection Notes'], dtype='object')

Updated DataFrame with Gazmazk4ez flag:


,Artist,Title,In Gazmazk4ez Collection
0,aaliyah,try again (a cappella),False
1,Absolute Zero & Subphonics,The Code,False
2,Adam F,Brand New Funk,False
3,Alix Perez,Down The Line (feat. MC Fats),False
4,Alix Perez,Fade Away,False
5,Alix Perez,Forsaken feat. Peven Everett & SpectraSoul,False
6,Alix Perez,Revolve-Her,False
7,Alix Perez,Never Left,False
8,Alix Perez,The Cut Deepens ft. Foreign Beggars,False
9,Ancronix,Skin it Back,False


In [10]:
# Update the matching logic to include both Artist and Title
songs_df['In Gazmazk4ez Collection'] = songs_df.apply(
    lambda row: any(
        (row['Title'].strip().lower() == gaz_title.strip().lower() and
         row['Artist'].strip().lower() == gaz_artist.strip().lower())
        for gaz_title, gaz_artist in zip(gazmazk4ez_df['Title'], gazmazk4ez_df['Artist'])
    ), axis=1
)

# Display the updated DataFrame
print("\nUpdated DataFrame with improved Gazmazk4ez flag:")
display(songs_df[['Artist', 'Title', 'In Gazmazk4ez Collection']].head(10))


Updated DataFrame with improved Gazmazk4ez flag:


,Artist,Title,In Gazmazk4ez Collection
0,aaliyah,try again (a cappella),False
1,Absolute Zero & Subphonics,The Code,False
2,Adam F,Brand New Funk,False
3,Alix Perez,Down The Line (feat. MC Fats),False
4,Alix Perez,Fade Away,False
5,Alix Perez,Forsaken feat. Peven Everett & SpectraSoul,False
6,Alix Perez,Revolve-Her,False
7,Alix Perez,Never Left,False
8,Alix Perez,The Cut Deepens ft. Foreign Beggars,False
9,Ancronix,Skin it Back,False


In [9]:
# Analyze the number of songs in the gazmazk4ez collection
num_in_collection = songs_df['In Gazmazk4ez Collection'].sum()
num_not_in_collection = len(songs_df) - num_in_collection

print(f"Number of songs in Gazmazk4ez collection: {num_in_collection}")
print(f"Number of songs not in Gazmazk4ez collection: {num_not_in_collection}")

# Display percentage
percentage_in_collection = (num_in_collection / len(songs_df)) * 100
print(f"Percentage of songs in Gazmazk4ez collection: {percentage_in_collection:.2f}%")

Number of songs in Gazmazk4ez collection: 24
Number of songs not in Gazmazk4ez collection: 2979
Percentage of songs in Gazmazk4ez collection: 0.80%


In [4]:
# Set up Discogs API integration
import os
import requests
from dotenv import load_dotenv

# Load the API key from the .env file
load_dotenv()
DISCOGS_API_KEY = os.getenv('DISCOGS_API_KEY')
if not DISCOGS_API_KEY:
    raise ValueError("Discogs API key not found in .env file")

# Define a function to search for songs by name using the Discogs API
def search_song_on_discogs(song_name, artist_name=None):
    """
    Search for a song on Discogs by name and optionally by artist.

    Parameters:
    song_name (str): The name of the song to search for.
    artist_name (str): The name of the artist (optional).

    Returns:
    dict: The JSON response from the Discogs API.
    """
    base_url = "https://api.discogs.com/database/search"
    headers = {
        'Authorization': f'Discogs token={DISCOGS_API_KEY}'
    }
    params = {
        'track': song_name,
        'type': 'release',
        'format': 'vinyl',  # Filter by format (vinyl)
        'per_page': 5,
        'page': 1
    }
    if artist_name:
        params['artist'] = artist_name

    response = requests.get(base_url, headers=headers, params=params)
    if response.status_code != 200:
        raise Exception(f"Discogs API request failed: {response.status_code} {response.text}")

    return response.json()

# Example usage
song_to_search = "Revolve-Her"
artist_to_search = "Alix Perez"
result = search_song_on_discogs(song_to_search, artist_to_search)

# Display the search results
print(f"Search results for '{song_to_search}' by '{artist_to_search}':")
for release in result.get('results', []):
    print(f"- Title: {release.get('title')}, Year: {release.get('year')}, Label: {release.get('label')}, Format: {release.get('format')}, Catalog#: {release.get('catno')}")

Search results for 'Revolve-Her' by 'Alix Perez':
- Title: Alix Perez - Recall And Reflect EP, Year: 2015, Label: ['Exit Records'], Format: ['Vinyl', '12"', 'Test Pressing'], Catalog#: EXIT059
- Title: Alix Perez - Recall And Reflect EP, Year: 2015, Label: ['Exit Records', 'Exit Records', 'Exit Records', 'Ten Eight Seven Mastering', 'Optimal Media GmbH'], Format: ['Vinyl', '12"', '45 RPM', 'EP'], Catalog#: EXIT 059
